In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import warnings

In [2]:
# Read File 
file_path = Path("Resources/Data/cleaned_news_df.csv")
news_df = pd.read_csv(file_path, header=0, encoding='utf-8')
# Display sample data
news_df.head(10)

,id,title,text,label
0,8476.0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294.0,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608.0,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142.0,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875.0,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903.0,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341.0,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95.0,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869.0,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909.0,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [3]:
#Raw csv file from Github
# news_data = "https://raw.githubusercontent.com/ml-truth-serum/fake_or_real_news/main/Resources/Data/news-jl.csv"

# # Read File
# news_df = pd.read_csv(news_data, header=0, encoding='utf-8')
# # Display sample data
# news_df.head(10)


## Step 1 - Clean the data

In [4]:
#check count of all rows
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6315 entries, 0 to 6314
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      6310 non-null   float64
 1   title   6315 non-null   object 
 2   text    6315 non-null   object 
 3   label   6315 non-null   object 
dtypes: float64(1), object(3)
memory usage: 197.5+ KB


In [6]:
#drop the id column
filter_df = news_df.loc[:,["title", "text", "label"]]
filter_df.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


## Step 2 - Get the columns that we need by transforming the columns

In [7]:
# Splitting the 'TextColumn' for each row based on whitespace
filter_df['title_array'] = filter_df['title'].apply(lambda x: x.split())
filter_df['text_array'] = filter_df['text'].apply(lambda x: x.split())

#only grab the label and the new array columns
transform_array_df = filter_df[['label', 'title_array', 'text_array']]

transform_array_df.head()

,label,title_array,text_array
0,FAKE,"[You, Can, Smell, Hillary’s, Fear]","[Daniel, Greenfield,, a, Shillman, Journalism,..."
1,FAKE,"[Watch, The, Exact, Moment, Paul, Ryan, Commit...","[Google, Pinterest, Digg, Linkedin, Reddit, St..."
2,REAL,"[Kerry, to, go, to, Paris, in, gesture, of, sy...","[U.S., Secretary, of, State, John, F., Kerry, ..."
3,FAKE,"[Bernie, supporters, on, Twitter, erupt, in, a...","[—, Kaydee, King, (@KaydeeKing), November, 9,,..."
4,REAL,"[The, Battle, of, New, York:, Why, This, Prima...","[It's, primary, day, in, New, York, and, front..."


In [15]:
transform_array_df = transform_array_df.astype(dtype={'label': 'string'})

In [16]:
#join array together

join_array_df = transform_array_df.loc[:,["label"]]

# Combine the arrays from both columns into a single array
join_array_df['combo_array'] = transform_array_df.apply(lambda row: np.concatenate([row['title_array'], row['text_array']]), axis=1)


join_array_df.head()

,label,combo_array
0,FAKE,"[You, Can, Smell, Hillary’s, Fear, Daniel, Gre..."
1,FAKE,"[Watch, The, Exact, Moment, Paul, Ryan, Commit..."
2,REAL,"[Kerry, to, go, to, Paris, in, gesture, of, sy..."
3,FAKE,"[Bernie, supporters, on, Twitter, erupt, in, a..."
4,REAL,"[The, Battle, of, New, York:, Why, This, Prima..."


### Title and Label Columns ONLY

In [17]:
#expand only the title array into new columns
title_expand_array_df = transform_array_df.loc[:,["label"]]

# Create new columns for each element in the arrays
# title_array_columns = transform_array_df['title_array'].apply(lambda x: pd.Series(x, dtype='object'))
#creats the columns as string dtype
title_array_columns = pd.DataFrame(transform_array_df['title_array'].tolist(), dtype='string')


# Concatenate the new columns with the original DataFrame
title_expand_array_df = pd.concat([title_expand_array_df, title_array_columns], axis=1)

#fill in na to "none"
title_expand_array_df = title_expand_array_df.fillna("NA")


title_expand_array_df.head()

,label,0,1,2,3,4,5,6,7,8,...,43,44,45,46,47,48,49,50,51,52
0,FAKE,You,Can,Smell,Hillary’s,Fear,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,FAKE,Watch,The,Exact,Moment,Paul,Ryan,Committed,Political,Suicide,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,REAL,Kerry,to,go,to,Paris,in,gesture,of,sympathy,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,FAKE,Bernie,supporters,on,Twitter,erupt,in,anger,against,the,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,REAL,The,Battle,of,New,York:,Why,This,Primary,Matters,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [18]:
title_expand_array_df.dtypes

label    string
0        string
1        string
2        string
3        string
4        string
5        string
6        string
7        string
8        string
9        string
10       string
11       string
12       string
13       string
14       string
15       string
16       string
17       string
18       string
19       string
20       string
21       string
22       string
23       string
24       string
25       string
26       string
27       string
28       string
29       string
30       string
31       string
32       string
33       string
34       string
35       string
36       string
37       string
38       string
39       string
40       string
41       string
42       string
43       string
44       string
45       string
46       string
47       string
48       string
49       string
50       string
51       string
52       string
dtype: object

### Text and Label Columns ONLY

In [19]:
#expand only the title array into new columns
text_expand_array_df = transform_array_df.loc[:,["label"]]

# Create new columns for each element in the arrays
text_array_columns = transform_array_df['text_array'].apply(lambda x: pd.Series(x, dtype='string'))

# Concatenate the new columns with the original DataFrame
text_expand_array_df = pd.concat([text_expand_array_df, text_array_columns], axis=1)

#fill in na to "none"
text_expand_array_df = text_expand_array_df.fillna("NA")

text_expand_array_df.head()

,label,0,1,2,3,4,5,6,7,8,...,5510,5511,5512,5513,5514,5515,5516,5517,5518,5519
0,FAKE,Daniel,"Greenfield,",a,Shillman,Journalism,Fellow,at,the,Freedom,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,FAKE,Google,Pinterest,Digg,Linkedin,Reddit,Stumbleupon,Print,Delicious,Pocket,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,REAL,U.S.,Secretary,of,State,John,F.,Kerry,said,Monday,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,FAKE,—,Kaydee,King,(@KaydeeKing),November,"9,",2016,The,lesson,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,REAL,It's,primary,day,in,New,York,and,front-runners,Hillary,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


### Combo(text and title) and Label Columns ONLY

In [20]:
#expand only the combo array into new columns
combo_expand_array_df = join_array_df.loc[:,["label"]]

# Create new columns for each element in the arrays
combo_array_columns = join_array_df['combo_array'].apply(lambda x: pd.Series(x, dtype='string'))

# Concatenate the new columns with the original DataFrame
combo_expand_array_df = pd.concat([combo_expand_array_df, combo_array_columns], axis=1)

#fill in na to "none"
combo_expand_array_df = combo_expand_array_df.fillna("")

combo_expand_array_df.head()

,label,0,1,2,3,4,5,6,7,8,...,5516,5517,5518,5519,5520,5521,5522,5523,5524,5525
0,FAKE,You,Can,Smell,Hillary’s,Fear,Daniel,"Greenfield,",a,Shillman,...,,,,,,,,,,
1,FAKE,Watch,The,Exact,Moment,Paul,Ryan,Committed,Political,Suicide,...,,,,,,,,,,
2,REAL,Kerry,to,go,to,Paris,in,gesture,of,sympathy,...,,,,,,,,,,
3,FAKE,Bernie,supporters,on,Twitter,erupt,in,anger,against,the,...,,,,,,,,,,
4,REAL,The,Battle,of,New,York:,Why,This,Primary,Matters,...,,,,,,,,,,


## Step 3 -  Prep and trained data 

In [31]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


# Drop 'label' column for X and assign 'label' column to y
X = combo_expand_array_df.drop('label', axis=1)
y = combo_expand_array_df['label']

# Create a column transformer with a pipeline for one-hot encoding each column
transformers = []

for i in range(len(X.columns)):
    transformers.append((str(i), Pipeline([('onehot', OneHotEncoder())]), [i]))

ct = ColumnTransformer(transformers=transformers, remainder='passthrough')

# Fit and transform X
X_transformed = ct.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=1/3, random_state=0)

In [32]:
print(X)

            0           1      2          3          4       5     \
0            You         Can  Smell  Hillary’s       Fear  Daniel   
1          Watch         The  Exact     Moment       Paul    Ryan   
2          Kerry          to     go         to      Paris      in   
3         Bernie  supporters     on    Twitter      erupt      in   
4            The      Battle     of        New      York:     Why   
...          ...         ...    ...        ...        ...     ...   
6310       State  Department   says         it      can't    find   
6311         The         ‘P’     in        PBS     Should   Stand   
6312  Anti-Trump  Protesters    Are      Tools         of     the   
6313          In   Ethiopia,  Obama      seeks   progress      on   
6314         Jeb        Bush     Is   Suddenly  Attacking  Trump.   

             6              7            8           9     ... 5516 5517 5518  \
0     Greenfield,              a     Shillman  Journalism  ...                  
1       C

In [33]:
print(y)

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6310    REAL
6311    FAKE
6312    FAKE
6313    REAL
6314    REAL
Name: label, Length: 6315, dtype: string


### Feature Scaling

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [35]:
print(X_train)

  (0, 368)	29.034482758620694
  (0, 2534)	32.45767938442841
  (0, 7047)	45.89118080072871
  (0, 9196)	64.89222016225153
  (0, 12190)	16.78303436068717
  (0, 18539)	64.89222016225153
  (0, 18767)	19.589024794476707
  (0, 23554)	9.833228354497244
  (0, 25405)	64.89222016225153
  (0, 28809)	45.89118080072871
  (0, 31940)	45.89118080072871
  (0, 34340)	26.50788738996085
  (0, 37907)	64.89222016225153
  (0, 41033)	20.54269653556662
  (0, 43799)	45.89118080072871
  (0, 48853)	26.50788738996085
  (0, 49401)	64.89222016225153
  (0, 52647)	18.023574875609828
  (0, 55493)	64.89222016225153
  (0, 58736)	64.89222016225153
  (0, 60859)	64.89222016225153
  (0, 63432)	64.89222016225153
  (0, 66407)	64.89222016225153
  (0, 71888)	6.803778837405602
  (0, 72858)	45.89118080072871
  :	:
  (4209, 2588980)	45.89118080072871
  (4209, 2588983)	45.89118080072871
  (4209, 2588986)	45.89118080072871
  (4209, 2588989)	45.89118080072871
  (4209, 2588992)	45.89118080072871
  (4209, 2588995)	45.89118080072871
  (42

In [36]:
print(X_test)

  (0, 2124)	1.0
  (0, 4097)	13.015605007668725
  (0, 7283)	22.961966463160493
  (0, 11318)	11.696746114005261
  (0, 14873)	1.0
  (0, 15538)	1.0
  (0, 20992)	19.589024794476707
  (0, 23810)	1.0
  (0, 27129)	1.0
  (0, 29221)	21.651326059270318
  (0, 31251)	45.89118080072871
  (0, 34238)	12.090465248814164
  (0, 37187)	20.54269653556662
  (0, 40652)	64.89222016225153
  (0, 44191)	64.89222016225153
  (0, 46850)	21.651326059270318
  (0, 48935)	18.75729702908665
  (0, 52791)	1.0
  (0, 55516)	1.0
  (0, 57582)	32.45767938442841
  (0, 63190)	45.89118080072871
  (0, 64765)	16.25204033264045
  (0, 66859)	37.474445245134085
  (0, 69411)	45.89118080072871
  (0, 72133)	32.45767938442841
  :	:
  (2104, 2588980)	45.89118080072871
  (2104, 2588983)	45.89118080072871
  (2104, 2588986)	45.89118080072871
  (2104, 2588989)	45.89118080072871
  (2104, 2588992)	45.89118080072871
  (2104, 2588995)	45.89118080072871
  (2104, 2588998)	45.89118080072871
  (2104, 2589001)	45.89118080072871
  (2104, 2589003)	45.891

### Training the Logistic Regression model on the Training set

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=1000, random_state = 0)
classifier.fit(X_train, y_train)

### Predicting the Test set results

In [ ]:
# Make predictions on the test set
y_pred = classifier.predict(X_test)

### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

### Visualising the Results

In [ ]:
import seaborn as sns
# Create a DataFrame with actual and predicted labels
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Create a confusion matrix
conf_matrix = confusion_matrix(results_df['Actual'], results_df['Predicted'])

# Map the actual and predicted labels to "Correctly Predicted" and "Incorrectly Predicted"
results_df['Outcome'] = results_df.apply(lambda row: 'Correctly Predicted' if row['Actual'] == row['Predicted'] else 'Incorrectly Predicted', axis=1)

# Plot the count of correct and incorrect predictions
plt.figure(figsize=(8, 6))
sns.countplot(x='Outcome', data=results_df, palette='Set2')

plt.title('Count of Correct and Incorrect Predictions')
plt.xlabel('Prediction Outcome')
plt.ylabel('Count')
plt.show()

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')